<a href="https://colab.research.google.com/github/andrebelem/pythonverse/blob/main/%5BPUBLIC%5D_Acessando_dados_climaticos_com_o_ERA5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](http://observatoriooceanografico.org/wp-content/uploads/elementor/thumbs/logo-o2-2-p31vla9ib17i85olifd4l1xk0e5lktrkal6u0s4c6o.png)<br>
# Como ler dados do ERA5 diretamente no Python (via API)
### Um exemplo voltado para o tratamento de dados climáticos
#### Plataforma Preferencial: Jupyter Notebook/Jupyter Lab/COLAB.
<br>

Criado por [A.L. Belém](mailto:andrebelem@id.uff.br) @ [Observatório Oceanográfico](http://www.observatoiriooceanografico.uff.br) 2023 (versão 2) para os colaboradores do O2 (e para quem mais se interessar)<br>
[![DOI](https://zenodo.org/badge/342919527.svg)](https://zenodo.org/badge/latestdoi/342919527)

Histórico:<br>
[12 de abril de 2021 Versão 1] criação do exemplo<br>
[28 de maio de 2021] Adaptado para o COLAB<br>
[18 de maio de 2023] Versão corrigida para usar o cartopy<br>


**NOTA: Caso você modifique esse notebook, tenha cuidado para não distribuir seus dados sigilosos de acesso ao CDS. Verifique antes de distribuir**

Este notebook foi *estimulado* pela participação no curso de **Introdução ao Python Científico** ministrado pelo meteorologista [Willy Hagi](https://linktr.ee/willyhagi) da [Meteonorte](https://www.meteonorte.com/) entre os dias 10 e 11 de abril de 2021, e baseado nas *lições aprendidas* a partir do documento de [Luke Gloege](https://lukegloege.medium.com/) no [Towards data science](https://towardsdatascience.com/read-era5-directly-into-memory-with-python-511a2740bba0) sobre o [ERA5](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview).

A ideia aqui é misturar o ERA5 e o [Cartopy](https://scitools.org.uk/cartopy/docs/v0.13/index.html). O `CartoPy` é um pacote criado pelo **UK Met Office** com o objetivo de permitir projeções cartográficas e plotagem de diversos tipos de dados geoespaciais em Python. **Nota COLAB**: Existe uma série de dificuldades para rodar alguns destes pacotes no COLAB, mas com um pouco de paciência, você pode configurar sua máquina virtual para (re)instalar os pacotes necessários. Veja abaixo ao longo do texto.

O ERA5 é a uma reanálise climática de quinta geração produzida pelo Centro Europeu de Previsões Meteorológicas de Médio Prazo (ECMWF) e ele fornece uma saída horária para muitos parâmetros atmosféricos, da superfície terrestre e do oceano, juntamente com algumas estimativas de incerteza. O volume de dados é imenso, bem como sua potencialidade, o que torna quase inviável o trabalho rotineiro com dados locais. Trabalhar na nuvem é a saida para não ocupar espaço e consumir banda de internet (o que é algo bastante incoerente para quem se preocupa com o clima x energia). Este notebook foi criado como uma "receita de bolo" para a instalação da API de armazenamento de dados climáticos (CDS), que irá permitir a leitura e análise de dados do ERA5 dentro do Python.

### O básico do ERA5 e do CDS API

O acesso via CDS API permite o trabalho sobre conjuntos de dados do ERA5 e do ERA5-land (apenas sobre terra). Existem quatro subconjuntos principais do ERA5: produtos horários e mensais, cada um em níveis de pressão (incluindo campos de ar em altitude) e níveis simples ou `single levels` (quantidades atmosféricas, oceânicas e da superfície terrestre). O ERA5-land inclui apenas níveis únicos. Se você é apressado, então ...
Os quatro subconjuntos ERA5 são:
- Dados horários ERA5 em níveis únicos
- Dados médios mensais ERA5 em níveis individuais
- Dados horários ERA5 sobre os níveis de pressão
- Dados médios mensais ERA5 sobre os níveis de pressão

Os dois subconjuntos ERA5-land:
- Dados horários ERA5-land
- Dados mensais do terreno ERA5

A reanálise ERA5 é descrita detalhadamente no paper de Hersbach, H, Bell, B, Berrisford, P, et al. The ERA5 global reanalysis. Q J R Meteorol Soc. 2020; 146: 1999– 2049. https://doi.org/10.1002/qj.3803

#### Primeiro, reconfiguração da sua máquina<br>

**Esses passos abaixo são exclusivamente para o COLAB**<br>

Para você ter todos os pacotes que precisa, vamos (1) atualizar os pacotes da máquina e instalar o que precisa, (2) resolver a briga entre o `shapely`e o `cartopy`, e (3) instalar o `cdsapi` para acessar os dados.<br>
**Atenção:** essa operação pode demorar de 2 à 3 minutos. Pegue um café e tenha um pouco de paciência.

In [ ]:
%%capture
# tudo isso aqui vamos usar. Demora um pouquinho para a máquina virtual criar tudo (~2 minutos). Tenha paciência
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cartopy
!pip uninstall -y shapely    # cartopy e shapely não são amigos ! (desde 2020)
!pip install shapely --no-binary shapely

#### Segundo, o acesso ao API do CDS

Nada vai acontecer se você não instalar a chave de API do CDS e para isso você precisa criar uma conta no Copernicus e armazenar sua chave API exclusiva em um arquivo dot. Aqui estão as etapas para fazer isso:
1. Cria uma conta no Copernicus [aqui](https://cds.climate.copernicus.eu/)
2. Depois de criada, entre na sua conta no Copernicus ([aqui](https://cds.climate.copernicus.eu/user)), e anote o `UID`e o `API key`que aparece no final da página se você acabou de criar OU clique sobre o seu nome de usuário na página principal e note que ao final da página estará essa informação. Sugiro que você copie-cole em um notepad ou similar. 
3. Modifique o *code snippet* abaixo usando um editor de texto, modificando os valores de `<UID>` e `<API key>` com os que você gravou.

'url: https://cds.climate.copernicus.eu/api/v2'<br>
'key: \<UID\>:\<API key\>'<br>
'verify: 0'<br>


**Note** que o nome do arquivo deve ser `.cdsapirc` e ele deve ser criado na sua `$HOME` (independente se é no windows ou no linux). **(Nota especial para o COLAB!)**: O código abaixo serve para você garantir que seus dados de chave estarão na home da máquina virtual. Porém, **cuidado** se você for compartilhar esse notebook na rede pois ele contém a sua chave pessoal de acesso ao CDS. 

Você pode criar diretamente no terminal do linux ou via notepad no Windows. Como verificação de integridade (e sua sanidade mental), verifique o conteudo do arquivo `.cdsapirc` para garantir que tudo pareça correto. Um arquivo correto será semelhante a este aqui:

'url: https://cds.climate.copernicus.eu/api/v2'<br>
'key: 54321:a77a7a7a-7a77-7777-7777-aa77a7a77a7a'<br>
'verify: 0'<br>

(**Obviamente** não copie o conteudo do arquivo acima como seu porque não irá funcionar).
<br>

In [ ]:
# Se você usa o Jupyter, crie o arquivo .crsapirc "por fora" e pule essa etapa
# EXCLUSIVO COLAB ....criação da chave no home do COLAB
###### altere os dados para sua própria chave ######
!echo url: https://cds.climate.copernicus.eu/api/v2 > ~/.cdsapirc
!echo key: 54321:a77a7a7a-7a77-7777-7777-aa77a7a77a7a >> ~/.cdsapirc
!echo verify:0 >> ~/.cdsapirc
# e verifica se está tudo bem
!more ~/.cdsapirc

#### Segundo passo, instale o cliente CDS API

Algo que não foi mencionado antes mas é bastante salutar, é você usar um *ambiente virtual* ((veja aqui)[https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/environments.html]). Independente disso, a instalação do cliente `cdsapi`é feiot via `conda`ou `pip`:

`pip install cdsapi` (Principalmente no COLAB)
ou 
`conda install cdsapi`

Depois de instalado, certifique-se de que você pode importar e iniciar o cliente em um script Python sem erros, por exemplo crianto um pequeno programa ou executando a linha abaixo.
<br>

In [ ]:
%%capture
# Exclusivo COLAB
# Instale agora a api do CDS para acesso.
!pip install cdsapi

In [ ]:
# e por fim, importe o seu cliente para baixar o ERA5
# Rode esse comando em qualquer ambiente, independente se é COLAB ou Jupyter
import cdsapi
cds = cdsapi.Client()

#### Agora ficou fácil! usando o cliente CDS API

O código abaixo mostra como solicitar temperaturas médias mensais em um nível de pressão especificado ao longo de um intervalo de datas. A saída é armazenada na memória como um `xarray` dataset e pode ser ativada ou desativada usando o `download_flag`:

In [ ]:
import warnings ## melhor ativar isso se você não quer encher sua tela com menssagens
warnings.filterwarnings("ignore")

In [ ]:
import cdsapi
import xarray as xr
from urllib.request import urlopen
# start the client
cds = cdsapi.Client()
# dataset you want to read
dataset = "reanalysis-era5-pressure-levels-monthly-means"
# flag to download data
download_flag = False
# api parameters 
params = {
    "format": "netcdf",
    "product_type": "monthly_averaged_reanalysis",
    "variable": "temperature",
    "pressure_level": '1000',
    'year':['2019','2020'],
    'month':['01','02','03'],
    "time": "00:00", 
    "grid": [1.0, 1.0],
    "area": [90, -180, -90, 180],
    }
# retrieves the path to the file
fl = cds.retrieve(dataset, params)
# download the file 
if download_flag:
    fl.download("./output.nc")
# load into memory
with urlopen(fl.location) as f:
    dset = xr.open_dataset(f.read())

In [ ]:
dset # para você dar uma olhadinha no que você baixou

In [ ]:
# note que você pode fazer experiências. Alguns campos tem 4 dimensões e outros 3. Adapte abaixo dependendo da sua necessidade;
#air = dset['t'][-1,1,:,:] -273.15
air = dset['t'][-1,:,:]-273.15

E agora podemos usar um código simples com o `cartopy` para plotar projeções com os campos globais de Temperatura do Ar (t). Note que a temperatura está em Kelvin e por isso fiz a conversão acima para °C. Outro ponto relevante é que pode aparecer aqui uma dimensão `expver` (**1** ou **5**) que são as versões do ERA5 (`expver 1`) e ERA5-Land (`expver 5`) (veja explicação (aqui)[https://confluence.ecmwf.int/pages/viewpage.action?pageId=173385064]). Se necessário você precisará fazer uma pré-seleção antes de plotar. Modifique se necessário.

In [ ]:
##### e agora uma linda receita de bolo #######
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE

plt.figure(figsize=(14,6))
ax = plt.axes(projection=ccrs.PlateCarree())
######### AQUI é que eu entro com o que eu quero plotar
air.plot(ax=ax,transform=ccrs.PlateCarree(),
         x='longitude',y='latitude',
         cmap='rainbow',add_colorbar=True)
####################################################
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)

paises = NaturalEarthFeature(category='cultural',scale='50m',facecolor='none',
                               name='admin_0_countries')

ax.add_feature(paises,facecolor='white',edgecolor='black')

ax.coastlines()

No lugar dos parâmetros de ano, mês, dia e hora, você pode, alternativamente, especificar uma lista de datas:

`'date':['2020-01-01 12:00', '2020-02-01 12:00', '2020-03-01 12:00'],`

ou você pode especificar uma data e hora como esta:

`'date':['2020-01-01', '2020-02-01', '2020-03-01'],`
`'time: '12:00',                                   `

Tanto faz você usar uma ou outra, o resultado é o mesmo na solicitação para a API. Essas solicitações são muito específicas e na maioria das vezes você pode solicitar um intervalo contínuo, como por exemplo `'date':'2020-01-01 12:00/2020-03-01 12:00'`.

Você ainda pode usar o pandas como um `date_range`, mas observe que a entrada para a API deve ser uma lista de *strings*. Aqui está um exemplo de uso de pandas:

`‘date’: list(pd.date_range(‘2020–01–01 12:00',’2020–03–01 12:00', freq=’M’).strftime(‘%Y-%m-%d %H:%M’)),` (**cautela com as aspas**).
<br>

#### Mais dicas úteis

**Mensagens de erro** <br>
Se nenhum dado está disponível no subconjunto solicitado e a solicitação não retornou dados, vai aparecer uma msg de erro.<br>
`no data is available within your requested subset. Request returned no data`
Isso geralmente indica que um valor de palavra-chave está fora do intervalo disponível. Por exemplo, apenas um conjunto específico de níveis de pressão pode ser solicitado. Além disso, o parâmetro de tempo deve ser para a hora mais próxima usando um relógio de 24 horas. Veja abaixo os erros comuns que vejo e como corrigi-los:

- O nível de pressão deve estar contido em: `[1, 2, 3, 5, 7, 10, 20, 30, 50, 70, range(100, 1000, 25)]`, onde a faixa (100,1000,25) indica 100 a 1000 em incrementos de 25. Se você receber um erro, verifique se a pressão que você está solicitando está contida nesta lista.
- O tempo deve corresponder à hora mais próxima em um relógio de 24 horas: `00:00, 01:00,…, 23:00`. Se você receber um erro, verifique se a hora é a hora mais próxima. Se você estiver solicitando médias mensais, certifique-se de que o parâmetro de tempo esteja definido como `‘00:00’`.

Existe um enorme [fórum](https://confluence.ecmwf.int/display/CUSF/forum) do CDS API e você pode perguntar suas dúvidas por lá. Não há uma documentação extensa do CDS API mas muitas informações podem ser obtidas [por aqui](https://cds.climate.copernicus.eu/api-how-to). Parte das palavras-chave permitidas e os nomes de variáveis variam dependendo do conjunto de dados. Os links a seguir são úteis:
- Os parâmetros de palavra-chave permitidos para cada conjunto de dados estão [neste link](https://confluence.ecmwf.int/display/CKB/Climate+Data+Store+%28CDS%29+API+Keywords).
- As variáveis disponíveis estão listadas em [tabelas como esta](https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation). Você fornece o que está listado no nome da variável na coluna CDS.
- Você também pode consultar o script de exemplo [aqui](https://confluence.ecmwf.int/display/CKB/Climate+Data+Store+(CDS)+API+Keywords).

Outra opção é usar a interface da web para fazer uma seleção e ver “mostrar solicitação de API” na parte inferior. Por exemplo, use [este link](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-monthly-means?tab=form) para escolher a "**média mensal dos níveis de pressão**". Outros conjuntos de dados podem ser encontrados [aqui](https://cds.climate.copernicus.eu/#!/search?text=ERA5&type=dataset). Clique no conjunto de dados que deseja acessar e, em seguida, na guia “Baixar dados” (ou `download data`) para fazer sua seleção. *O céu (ou o oceano) é o limite*.


#### Jupyterize seus scripts tornando a API mais fácil de usar

Quando você entender como funciona, você percebe que a API do CDS é extremamente muito poderosa. Compartilhe com seus colegas, "pythonize-se". Todas as informações aqui são de código aberto. Fique à vontade para copiá-la e modificá-la como desejar. Se usar o COLAB, melhor ainda porque dai qualquer pessoa mesmoooo poderá usar, em qualquer lugar.

<img src='https://www.python.org/static/community_logos/python-logo-master-v3-TM-flattened.png' width=150px></img>
<img src='https://cdn.iconscout.com/icon/free/png-256/love-chat-2125802-1789050.png' width=50px></img>
<img src='https://miro.medium.com/max/1400/1*7oukapIBInsovpHkQB3QZg.jpeg' width=150px></img>

